## Data Model Item in CosmosDB

```json
"id" : "uuid",
"filepath" : "Shared/document/somepermit.pdf",
"documentTitle" : "somepermit.pdf",
"documentSummary" : "summary for this document", // optional
"organization" : "KPI",
"subOrganization" : ["Sertifikat_PLO_SKPI", "PLO RU II"], // optional
"permitType" : "PLO",
"permits" : [
    {
        "permitNumber" : "94404/394040",
        "issueDate" : "2024-01-15T00:00:00Z",
        "expiryDate" : "2025-10-12T00:00:00Z",
        "permitSummary": "This permit is for ..." // optional
    },
    {
        "permitNumber" : "94404/394041",
        "issueDate" : "2023-01-15T00:00:00Z",
        "expiryDate" : "2024-01-15T00:00:00Z",
        "permitSummary" : "This permit is for ..."
    }
]
```

## Notes

**PLO**
- 1 Dokumen PLO bisa ada beberapa nomor. Dokumen kadang dituliskan dengan 
  - nomor-PLO 1 "s/d" nomor-PLO 2
  - nomor-PLO "-" nomor-PLO 2 / nomor belakang PLO
- Dokumen PLO dengan nomor berbeda memiliki `issue_date` dan `expiry_date` yang berbeda-beda
- Dokumen yang 1 dokumen 1 PLO, ada yang merupakan potongan dokumen
- Ada case dimana lebih dari 2 (atau 2) nomer PLO dan itu sudah berurutan.
- Ada nomor dokumen yang tidak ada "PLO"
- Ada 2 dokumen yang nomor PLO nya sama

**KKPR/KKPRL**
- Surat ijin lingkungan ada issue date. Tapi tidak di halaman awal-awal. Bisa di atas 10 lebih
- Tidak ada `expiry_date`

**Notes**
- Permit yang punya halaman banyak (lebih dari 74) adalah PLO, Persetujuan Lingkungan, dan Izin Lingkungan

In [1]:
import os
import io
import base64

from pydantic import BaseModel, Field
from typing import List, Any, Literal
from pdf2image import convert_from_path

from openai import AzureOpenAI

from dotenv import load_dotenv

load_dotenv()

api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

client = AzureOpenAI(
    base_url=f"{api_base}/openai/deployments/{deployment_name}",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

class PermitNumber(BaseModel):
    permit_type: Literal["PLO", "KKPR/KKPRL", "Ijin Lingkungan"] = Field(..., description="The type/category of the permit.")
    permit_number: List[str] = Field(..., description="The unique identifier for the permit.")


extraction_prompt = """
You are an expert in extracting metadata from permit images. Given an image of a permit, extract the following metadata fields:
- Permit Type. Permit type is categorized as PLO, KKPR/KKPRL, or Ijin Lingkungan. Only return one of these three types.
- Permit Number. 
  - Permit number is usually a combination of letters and numbers, often found at the top of the permit.
  - Permit number may contain multiple identifiers separated by commas.
  - One permit number may contain alphanumeric characters, slashes (/).
  - Return as string without any additional text and spaces.

Return the extracted metadata in the JSON format. Do not include any additional text or explanations.
"""

def base64_encoded_image(image, format: str = 'JPEG') -> str:
    """
    Generate a base64 encoded string from image object.
    
    Args:
        image: Image object (e.g., PIL.Image).
        
    Returns:
        str: Base64 encoded string of the image.
    """

    buffer = io.BytesIO()
    image.save(buffer, format=format)
    buffer.seek(0)
    encoded_str = base64.b64encode(buffer.read()).decode('utf-8')
    buffer.close()

    image_data = f"data:image/{format.lower()};base64,{encoded_str}"

    return image_data

def create_message(system_prompt: str, document_title: str, image_content: List[Any]) -> dict:
    """
    Create a message payload for the chat API.

    Args:
        system_prompt (str): The system prompt to include in the message.
        document_title (str): The title of the document being processed.
        image_content (List[Any]): The content to include in the message.

    Returns:
        dict: The message payload.
    """
    message = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": [
            {
                "type" : "text",
                "text": f"Extract the metadata from the permit image with this title: {document_title}."
            }
            ]
        }
    ]

    for content in image_content:
        message[1]['content'].append({"type": "image_url", "image_url": {"url" : base64_encoded_image(content)}})

    return message

In [128]:
import instructor

instructor_client = instructor.from_provider("azure_openai/gpt-4o")

list_of_files = [
    "data/folder/KPI_Permits/Sertifikat_PLO_SKPI/PLO RU II/PLO 9 Pipa Penyalur.pdf",
    "data/folder/PPN_Permits/Sertifikat_PLO/DPPU Sentani, Babullah, Pattimura, Mopah, Frans Kaisiepo, Rendani, DEO.pdf",
    "data/folder/KPI_Permits/Sertifikat_PLO_SKPI/PLO RU II/PLO CDU KILANG SEI PAKNING.pdf",
    "data/folder/PPN_Permits/KKPR/KKPR Stranas/19. RKKPR Stranas Jalur Pipa Cilacap - Yogyakarta (CY) 1, (CY) 2, dan (CY) 3.pdf",
    "data/folder/PPN_Permits/Persetujuan_Lingkungan/C JBB/A2. DPPU SHAFTHI/A2 DPPU SHAFTI Persetujuan Lingkungan (SK.11).pdf",
    "data/folder/KPI_Permits/Sertifikat_PLO_SKPI/PLO RU III/PLO RU III Plaju Sumsel_MASTER FILE PLO & COI 2022.pdf"
]

FILE_PATH = list_of_files[1]

print(f"Processing file: {FILE_PATH}")
images = convert_from_path(FILE_PATH)
print(f"Total pages converted to images: {len(images)}")
print("Using first 1 pages for metadata extraction.")
message_payload = create_message(extraction_prompt, FILE_PATH, images[:1])

print("Sending request to Azure OpenAI...")
user, completion = instructor_client.chat.completions.create_with_completion(
    messages=message_payload,
    response_model=PermitNumber
)

INFO:instructor.auto_client:Initializing azure_openai provider with model gpt-4o
INFO:instructor.auto_client:Client initialized


Processing file: data/folder/PPN_Permits/Sertifikat_PLO/DPPU Sentani, Babullah, Pattimura, Mopah, Frans Kaisiepo, Rendani, DEO.pdf
Total pages converted to images: 145
Using first 1 pages for metadata extraction.
Sending request to Azure OpenAI...


INFO:httpx:HTTP Request: POST https://openai-permit.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"


In [129]:
user

PermitNumber(permit_type='PLO', permit_number=['084-090/49-8/PLO/2024'])

In [87]:
for permit_number in user.permit_number:
    if "PLO" in permit_number:
        print(f"Found PLO Permit Number: {permit_number}")

Found PLO Permit Number: 079/48-3/PLO/DMT/2023
Found PLO Permit Number: 088/48-3/PLO/DMT/2023


In [88]:
print(user.permit_number)

if len(user.permit_number) > 1:
    print("Found multiple permit numbers")
    permit_last_number = "/".join(user.permit_number[0].split("/")[1:])
    start_number = int(user.permit_number[0].split("/")[0])
    end_number = int(user.permit_number[-1].split("/")[0])

    number_range = [f"{i:03d}" for i in range(start_number, end_number + 1)]
    result_string = ", ".join(number_range)
    for result in result_string.split(", "):
        print("/".join([result, permit_last_number]))

elif "-" in user.permit_number[0].split("/")[0]:
    print("Found permit number range")
    permit_last_number = "/".join(user.permit_number[0].split("/")[1:])
    start_number = int(user.permit_number[0].split("/")[0].split("-")[0])
    end_number = int(user.permit_number[0].split("/")[0].split("-")[1])

    number_range = [f"{i:03d}" for i in range(start_number, end_number + 1)]
    result_string = ", ".join(number_range)
    for result in result_string.split(", "):
        print("/".join([result, permit_last_number]))
else:
    print("Single permit number found:", user.permit_number)


['079/48-3/PLO/DMT/2023', '088/48-3/PLO/DMT/2023']
Found multiple permit numbers
079/48-3/PLO/DMT/2023
080/48-3/PLO/DMT/2023
081/48-3/PLO/DMT/2023
082/48-3/PLO/DMT/2023
083/48-3/PLO/DMT/2023
084/48-3/PLO/DMT/2023
085/48-3/PLO/DMT/2023
086/48-3/PLO/DMT/2023
087/48-3/PLO/DMT/2023
088/48-3/PLO/DMT/2023


## Extract Permit Type and Initial Permit Number

In [2]:
from PyPDF2 import PdfReader

file_pages = []

def count_pdf_pages(file_path):
    reader = PdfReader(file_path)
    return len(reader.pages)

for file_path in file_list:
    file_pages.append(count_pdf_pages(file_path))

NameError: name 'file_list' is not defined

In [107]:
import logging
import asyncio

from tenacity import retry, wait_exponential, stop_after_attempt, before_sleep_log

logging.basicConfig(level=logging.INFO)

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=4, max=10),
    before_sleep=before_sleep_log(logging.getLogger(), logging.WARNING)
)
async def get_permit_number(doc_path:str) -> PermitNumber:
    loop = asyncio.get_event_loop()
    images = await loop.run_in_executor(None, convert_from_path, doc_path)
    message_payload = create_message(extraction_prompt, doc_path, images[:1])

    user, completion = await loop.run_in_executor(
        None,
        instructor_client.chat.completions.create_with_completion,
        message_payload,
        PermitNumber
    )

    return {
        "filepath": doc_path,
        "permit_type": user.permit_type,
        "permit_number": user.permit_number,
        "usage": completion.usage
    }

list_of_files = [
    "data/folder/KPI_Permits/Sertifikat_PLO_SKPI/PLO RU II/PLO 9 Pipa Penyalur.pdf",
    "data/folder/PPN_Permits/Sertifikat_PLO/DPPU Sentani, Babullah, Pattimura, Mopah, Frans Kaisiepo, Rendani, DEO.pdf",
    "data/folder/KPI_Permits/Sertifikat_PLO_SKPI/PLO RU II/PLO CDU KILANG SEI PAKNING.pdf",
    "data/folder/PPN_Permits/KKPR/KKPR Stranas/19. RKKPR Stranas Jalur Pipa Cilacap - Yogyakarta (CY) 1, (CY) 2, dan (CY) 3.pdf",
    "data/folder/PPN_Permits/Persetujuan_Lingkungan/C JBB/A2. DPPU SHAFTHI/A2 DPPU SHAFTI Persetujuan Lingkungan (SK.11).pdf",
    "data/folder/KPI_Permits/Sertifikat_PLO_SKPI/PLO RU III/PLO RU III Plaju Sumsel_MASTER FILE PLO & COI 2022.pdf"
]

result = await get_permit_number(list_of_files[2])
print(result)

INFO:httpx:HTTP Request: POST https://openai-permit.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"


{'filepath': 'data/folder/KPI_Permits/Sertifikat_PLO_SKPI/PLO RU II/PLO CDU KILANG SEI PAKNING.pdf', 'permit_type': 'PLO', 'permit_number': ['521/48-4/PLO/DMT/2022'], 'usage': CompletionUsage(completion_tokens=25, prompt_tokens=1203, total_tokens=1228, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=None), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=1152))}


INFO:httpx:HTTP Request: POST https://openai-permit.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-permit.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-permit.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-permit.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-permit.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-permit.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-permit.openai.azure.com/openai/deploy

In [108]:
import os
from PyPDF2 import PdfReader

# Get all PDF files in the folder
folder_path = "data/folder"
file_list = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.lower().endswith(".pdf"):
            file_list.append(os.path.join(root, file))

print(file_list)

# Count pages in each PDF file
file_pages = []

def count_pdf_pages(file_path):
    reader = PdfReader(file_path)
    return len(reader.pages)

for file_path in file_list:
    file_pages.append(count_pdf_pages(file_path))

['data/folder/PGN_Permits/4. Perling_PGN/Persetujuan Lingkungan SOR 1/SK Persetujuan Lingkungan UKL-UPL Stasiun Gas Dumai.PDF', 'data/folder/PGN_Permits/4. Perling_PGN/PL Th 2023 - Jargas CGP/Perling - PKPLH  Jargas Kota Yogyakarta dan Kab Sleman.pdf', 'data/folder/PGN_Permits/4. Perling_PGN/Persetujuan Lingkungan PMO/PKPLH UKL UPL KIK-Mangkang, Kendal.pdf', 'data/folder/PGN_Permits/4. Perling_PGN/Persetujuan Lingkungan SOR 2/SK Persetujuan PKPLH Upgrading Offtake Garawangi.PDF', 'data/folder/PGN_Permits/4. Perling_PGN/Persetujuan Lingkungan SOR 2/SK Persetujuan PKPLH Upgrading Offtake Sunyaragi.PDF', 'data/folder/PGN_Permits/4. Perling_PGN/Persetujuan Lingkungan SOR 2/SK Persling UKL-UPL Cabang Jkt.pdf', 'data/folder/PGN_Permits/4. Perling_PGN/Persetujuan Lingkungan SOR 2/SK Persetujuan Lingkungan UKL-UPL Cabang Bogor.PDF', 'data/folder/PGN_Permits/4. Perling_PGN/Persetujuan Lingkungan SOR 2/SK Persetujuan SKKL AMDAL PDJB.PDF', 'data/folder/PGN_Permits/4. Perling_PGN/Persetujuan Lingk

In [109]:
permit_number_result = []

for file_path in file_list:
    result = await get_permit_number(file_path)
    permit_number_result.append(result)

INFO:httpx:HTTP Request: POST https://openai-permit.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-permit.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-permit.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-permit.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-permit.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-permit.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-permit.openai.azure.com/openai/deploy

In [113]:
import json

# Convert usage objects to dictionaries for JSON serialization
def serialize_metadata(metadata_list):
    serialized = []
    for item in metadata_list:
        serialized_item = item.copy()
        if 'usage' in serialized_item and serialized_item['usage'] is not None:
            serialized_item['usage'] = serialized_item['usage'].model_dump()
        serialized.append(serialized_item)
    return serialized

# Write metadata_result to JSON file
output_file = "data/document_details.json"

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(serialize_metadata(permit_number_result), f, indent=2, ensure_ascii=False)

print(f"Output saved to {output_file}")

Output saved to data/document_details.json


In [125]:
# Check PLO permit number yang tidak mengandung "PLO"
for item in permit_number_result:
    if item['permit_type'] == "PLO":
        for permit_number in item['permit_number']:
            check_plo = []
            if "PLO" in permit_number:
                status = True
                check_plo.append(True)
            else:
                check_plo.append(False)

        if sum(check_plo) < 1:
            print(item['permit_number'])
            print(f"Permit number not found in file: {item['filepath']}")

['88/48-5/SKPI/18.01/DJM.T/2018']
Permit number not found in file: data/folder/KPI_Permits/Sertifikat_PLO_SKPI/PLO RU V/1378_SKPI_SKPI OM DAN LAWE2 Feb 2018-Jan 2023.pdf
['87/48-5/SKPI/18.01/DJM.T/2018']
Permit number not found in file: data/folder/KPI_Permits/Sertifikat_PLO_SKPI/PLO RU V/1376_SKPI_SKPI HSC Feb 2018-Jan 2023.pdf
['86/48-5/SKPI/18.01/DJM.T/2018']
Permit number not found in file: data/folder/KPI_Permits/Sertifikat_PLO_SKPI/PLO RU V/1375_SKPI_SKPI HCC Feb 2018-Jan 2023.pdf


In [ ]:
# import duckdb

## Insert number of pages into the database
# con = duckdb.connect(database='data/metadata_document.db')

# for file_path, num_pages in zip(file_list, file_pages):
#     con.execute(f"""
#                 UPDATE document_details
#                 SET num_pages = '{num_pages}'
#                 WHERE filepath = '{file_path}';
#                 """)
# con.commit()
# con.close()

In [38]:
import duckdb

con = duckdb.connect(database='data/metadata_document.db', read_only=True)

query = """
select permit_type, 
    ROUND(avg(num_pages), 2) as avg,
    MIN(num_pages) as min,
    QUANTILE(num_pages, 0.25) as q25,
    QUANTILE(num_pages, 0.5) as q50,
    QUANTILE(num_pages, 0.75) as q75,
    MAX(num_pages) as max, 
    COUNT(*) as count,
    ROUND(STDDEV(num_pages),2) as stdev
  from document_details
  group by permit_type;
"""

result = con.sql(query)
print(result)
con.close()

┌─────────────────┬────────┬───────┬───────┬───────┬───────┬───────┬───────┬────────┐
│   permit_type   │  avg   │  min  │  q25  │  q50  │  q75  │  max  │ count │ stdev  │
│     varchar     │ double │ int32 │ int32 │ int32 │ int32 │ int32 │ int64 │ double │
├─────────────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┼────────┤
│ KKPR/KKPRL      │   6.06 │     2 │     4 │     4 │     8 │    20 │    98 │   3.41 │
│ PLO             │  71.71 │     1 │     5 │    14 │    94 │   636 │    82 │  114.3 │
│ Ijin Lingkungan │ 108.07 │    12 │    37 │    84 │   171 │   310 │    58 │  80.45 │
└─────────────────┴────────┴───────┴───────┴───────┴───────┴───────┴───────┴────────┘



In [1]:
import duckdb

con = duckdb.connect(database='data/metadata_document.db', read_only=True)

query = """
select count(*) as _count, permit_number from document_details 
  group by permit_number 
  order by _count desc limit 10
"""

result = con.sql(query)
print(result)
con.close()

┌────────┬────────────────────────────────────────────────┐
│ _count │                 permit_number                  │
│ int64  │                   varchar[]                    │
├────────┼────────────────────────────────────────────────┤
│     10 │ [079/48-3/PLO/DMT/2023, 088/48-3/PLO/DMT/2023] │
│      3 │ [SK.412/MENLHK/SETJEN/PLA.4/4/2022]            │
│      2 │ [PF.01/2075-200/XII/2023]                      │
│      2 │ [PF.01/2009-200/XII/2023]                      │
│      2 │ [PF.01/1967-200/XI/2023]                       │
│      2 │ [062/50/PLO/DMT/2023, 063/50/PLO/DMT/2023]     │
│      2 │ [SK.1019/MENLHK/SETJEN/PLA.4/11/2021]          │
│      2 │ [392/48-4/PLO/DMT/2024, 397/48-4/PLO/DMT/2024] │
│      1 │ [13032310211671024]                            │
│      1 │ [09082410511400005]                            │
├────────┴────────────────────────────────────────────────┤
│ 10 rows                                       2 columns │
└───────────────────────────────────────

In [6]:
from typing import List, Any, Literal
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("cleansing_plo_number")

plo_combination_test = [
    ["109/50/PLO/DMT/2023"], # single PLO
    ["062/50/PLO/DMT/2023", "063/50/PLO/DMT/2023"], # urutan + 1 dan lebih dari 2
    ["B-2191/MG.06/DMTO/2023", "139/50/PLO/DMT/2023"], # lebih dari 2 tapi hanya ada 1 PLO
    ["079/48-3/PLO/DMT/2023", "088/48-3/PLO/DMT/2023"], # lebih dari 2 tapi ada urutan (dokumen hasil iterasi lebih dari 1)
    ["B-1392/MG.06/DMTO/2023", "079/48-3/PLO/DMT/2023", "088/48-3/PLO/DMT/2023"],
    ["325/50/PLO/DMT/2024", "326/50/PLO/DMT/2024", "327/50/PLO/DMT/2024"], # lebih dari 2 dan urutan + 1
    ["084-090/49-8/PLO/2024"] # 1 dokumen, range nomor PLO dengan tanda "-"
]

def cleansing_plo_number(permit_number: List[str]) -> List[str]:
    """
    Cleansing PLO permit numbers to expand ranges and multiple entries.

    Args:
        permit_number (List[str]): List of permit numbers.
    """
    
    plo_numbers_found = []
    for item in permit_number:
        if "PLO" in item:
            plo_numbers_found.append(item)
        else:
            pass
    if len(plo_numbers_found) == 0:
        return []

    if len(plo_numbers_found) > 1:
        permit_last_number = "/".join(plo_numbers_found[0].split("/")[1:])
        start_number = int(plo_numbers_found[0].split("/")[0])
        end_number = int(plo_numbers_found[-1].split("/")[0])

        number_range = [f"{i:03d}" for i in range(start_number, end_number + 1)]
        result_string = ", ".join(number_range)
        plo_final_list = []
        for result in result_string.split(", "):
            plo_final_list.append("/".join([result, permit_last_number]))

        return plo_final_list

    elif "-" in plo_numbers_found[0].split("/")[0]:
        permit_last_number = "/".join(plo_numbers_found[0].split("/")[1:])
        start_number = int(plo_numbers_found[0].split("/")[0].split("-")[0])
        end_number = int(plo_numbers_found[0].split("/")[0].split("-")[1])

        number_range = [f"{i:03d}" for i in range(start_number, end_number + 1)]
        result_string = ", ".join(number_range)
        plo_final_list = []
        for result in result_string.split(", "):
            plo_final_list.append("/".join([result, permit_last_number]))

        return plo_final_list

    else:
        return plo_numbers_found

In [11]:
for plo_list in plo_combination_test:
    print("Input PLO Permit Numbers:", plo_list)
    cleaned_plo_numbers = cleansing_plo_number(plo_list)
    print("Cleansed PLO Permit Numbers:", cleaned_plo_numbers)
    print("-----")

Input PLO Permit Numbers: ['109/50/PLO/DMT/2023']
Cleansed PLO Permit Numbers: ['109/50/PLO/DMT/2023']
-----
Input PLO Permit Numbers: ['062/50/PLO/DMT/2023', '063/50/PLO/DMT/2023']
Cleansed PLO Permit Numbers: ['062/50/PLO/DMT/2023', '063/50/PLO/DMT/2023']
-----
Input PLO Permit Numbers: ['B-2191/MG.06/DMTO/2023', '139/50/PLO/DMT/2023']
Cleansed PLO Permit Numbers: ['139/50/PLO/DMT/2023']
-----
Input PLO Permit Numbers: ['079/48-3/PLO/DMT/2023', '088/48-3/PLO/DMT/2023']
Cleansed PLO Permit Numbers: ['079/48-3/PLO/DMT/2023', '080/48-3/PLO/DMT/2023', '081/48-3/PLO/DMT/2023', '082/48-3/PLO/DMT/2023', '083/48-3/PLO/DMT/2023', '084/48-3/PLO/DMT/2023', '085/48-3/PLO/DMT/2023', '086/48-3/PLO/DMT/2023', '087/48-3/PLO/DMT/2023', '088/48-3/PLO/DMT/2023']
-----
Input PLO Permit Numbers: ['B-1392/MG.06/DMTO/2023', '079/48-3/PLO/DMT/2023', '088/48-3/PLO/DMT/2023']
Cleansed PLO Permit Numbers: ['079/48-3/PLO/DMT/2023', '080/48-3/PLO/DMT/2023', '081/48-3/PLO/DMT/2023', '082/48-3/PLO/DMT/2023', '083/

In [3]:
import duckdb
import pandas as pd

con = duckdb.connect(database='data/metadata_document.db', read_only=True)

query = """
select filepath, permit_number from document_details
    where permit_type = 'PLO';
"""


df = con.execute(query).df()

con.close()

In [11]:
df

,filepath,permit_number
0,data/folder/PGN_Permits/3. PLO_PGN/PLO_Proyek_...,"[062/50/PLO/DMT/2023, 063/50/PLO/DMT/2023]"
1,data/folder/PGN_Permits/3. PLO_PGN/PLO_Proyek_...,[444/50/PLO/DMT/2022]
2,data/folder/PGN_Permits/3. PLO_PGN/PLO_Proyek_...,"[B-2191/MG.06/DMTO/2023, 139/50/PLO/DMT/2023]"
3,data/folder/PGN_Permits/3. PLO_PGN/PLO_Proyek_...,[109/50/PLO/DMT/2023]
4,data/folder/PGN_Permits/3. PLO_PGN/PLO_Proyek_...,"[B-9746/MG.06/DMTO/2022, 056300.S/HK.01/PMO/20..."
...,...,...
77,data/folder/PPN_Permits/Sertifikat_PLO/Sertifi...,[162/49-5/PLO/DMT/2024]
78,data/folder/PPN_Permits/Sertifikat_PLO/Sertifi...,[745/49/4/PLO/DMT/2022]
79,data/folder/PPN_Permits/Sertifikat_PLO/Sertifi...,[164/49-5/PLO/DMT/2024]
80,data/folder/PPN_Permits/Sertifikat_PLO/Sertifi...,[042/49-4/PLO/DMT/2024]


In [8]:
cleaned_plo_numbers_list = []

for plo_number in df['permit_number']:
    print("Original PLO Permit Numbers:", plo_number)
    cleaned_plo_numbers = cleansing_plo_number(plo_number)
    print("Cleansed PLO Permit Numbers:", cleaned_plo_numbers)
    cleaned_plo_numbers_list.append(cleaned_plo_numbers)


Original PLO Permit Numbers: ['062/50/PLO/DMT/2023' '063/50/PLO/DMT/2023']
Cleansed PLO Permit Numbers: ['062/50/PLO/DMT/2023', '063/50/PLO/DMT/2023']
Original PLO Permit Numbers: ['444/50/PLO/DMT/2022']
Cleansed PLO Permit Numbers: ['444/50/PLO/DMT/2022']
Original PLO Permit Numbers: ['B-2191/MG.06/DMTO/2023' '139/50/PLO/DMT/2023']
Cleansed PLO Permit Numbers: ['139/50/PLO/DMT/2023']
Original PLO Permit Numbers: ['109/50/PLO/DMT/2023']
Cleansed PLO Permit Numbers: ['109/50/PLO/DMT/2023']
Original PLO Permit Numbers: ['B-9746/MG.06/DMTO/2022' '056300.S/HK.01/PMO/2022' '412/50/PLO/DMT/2022']
Cleansed PLO Permit Numbers: ['412/50/PLO/DMT/2022']
Original PLO Permit Numbers: ['142/50/PLO/DMT/2023']
Cleansed PLO Permit Numbers: ['142/50/PLO/DMT/2023']
Original PLO Permit Numbers: ['092/50/PLO/DMT/2023']
Cleansed PLO Permit Numbers: ['092/50/PLO/DMT/2023']
Original PLO Permit Numbers: ['B-10285/MG.06/DMTO/2022' '514/50/PLO/DMT/2022']
Cleansed PLO Permit Numbers: ['514/50/PLO/DMT/2022']
Origi

In [9]:
cleaned_plo_numbers_list

[['062/50/PLO/DMT/2023', '063/50/PLO/DMT/2023'],
 ['444/50/PLO/DMT/2022'],
 ['139/50/PLO/DMT/2023'],
 ['109/50/PLO/DMT/2023'],
 ['412/50/PLO/DMT/2022'],
 ['142/50/PLO/DMT/2023'],
 ['092/50/PLO/DMT/2023'],
 ['514/50/PLO/DMT/2022'],
 ['062/50/PLO/DMT/2023', '063/50/PLO/DMT/2023'],
 ['110/50/PLO/DMT/2023'],
 ['129/50/PLO/DMT/2023'],
 ['494/50/PLO/DMT/2021'],
 ['417/50-3/PLO/DMT/2024'],
 ['419/50-3/PLO/DMT/2024'],
 ['420/50-3/PLO/DMT/2024'],
 ['326/50/PLO/DMT/2024'],
 ['327/50/PLO/DMT/2024'],
 ['418/50-3/PLO/DMT/2024', '419/50-3/PLO/DMT/2024', '420/50-3/PLO/DMT/2024'],
 ['325/50/PLO/DMT/2024', '326/50/PLO/DMT/2024', '327/50/PLO/DMT/2024'],
 ['053/50-3/PLO/DMT/2024'],
 ['036/48-2/PLO/DMT/2024',
  '037/48-2/PLO/DMT/2024',
  '038/48-2/PLO/DMT/2024',
  '039/48-2/PLO/DMT/2024',
  '040/48-2/PLO/DMT/2024',
  '041/48-2/PLO/DMT/2024',
  '042/48-2/PLO/DMT/2024',
  '043/48-2/PLO/DMT/2024',
  '044/48-2/PLO/DMT/2024'],
 ['521/48-4/PLO/DMT/2022'],
 ['749/48-2/PLO/DMT/2022'],
 ['522/48-4/PLO/DMT/2022'],


In [34]:
import sys
import os
import pytesseract

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from PIL import Image
from src.utils.data_ingestion import setup_logger
from typing import List

logger = setup_logger(level = logging.DEBUG)

def classify_with_keywords(images: List[Image.Image], anchor_keywords: List[str], min_score: int = 1) -> List[int]:
    """
    Runs OCR on each PIL Image and returns indices containing anchor keywords.
    Args:
        images (List[Image.Image]): List of PIL Image objects.
        anchor_keywords (List[str]): List of keywords to search for.
        min_score (int): Minimum number of keyword matches to consider a page as candidate.
    Returns:
        List[int]: Indices of images containing anchor keywords.
    """
    candidate_indices: List[int] = []

    for idx, img in enumerate(images):
        text: str = pytesseract.image_to_string(img)
        lines = list(map(lambda x: x.replace(" ", ""), text.split('\n')))
        detected_keywords = [kw for kw in anchor_keywords if any(kw.lower() in line.lower() for line in lines)]
        score: int = len(detected_keywords)
        if score >= min_score:
            candidate_indices.append(idx)
            logger.debug(f"Page {idx + 1}: Detected keywords: {detected_keywords}")

    return candidate_indices

In [39]:
from typing import Dict, List


def map_keywords_to_pages(
    images: List[Image.Image], 
    anchor_keywords: List[str]
) -> Dict[str, List[int]]:
    """
    Maps each anchor keyword to the page indices where it appears.
    
    Args:
        images (List[Image.Image]): List of PIL Image objects.
        anchor_keywords (List[str]): List of keywords to search for.
    
    Returns:
        Dict[str, List[int]]: Dictionary mapping each keyword to list of page indices where it appears.
    """
    keyword_to_pages: Dict[str, List[int]] = {kw: [] for kw in anchor_keywords}
    
    for idx, img in enumerate(images):
        text: str = pytesseract.image_to_string(img)
        lines = list(map(lambda x: x.replace(" ", ""), text.split('\n')))
        
        for kw in anchor_keywords:
            if any(kw.lower() in line.lower() for line in lines):
                keyword_to_pages[kw].append(idx)
                logger.debug(f"Found '{kw}' on page {idx + 1}")
    
    return keyword_to_pages

In [ ]:
images = convert_from_path(df['filepath'][17])
cleaned_plo_numbers = cleansing_plo_number(df['permit_number'][17])

result_indices = classify_with_keywords(
    images,
    anchor_keywords=cleaned_plo_numbers)

[2025-11-05 11:45:55,960] DEBUG: Page 1: Detected keywords: ['062/50/PLO/DMT/2023', '063/50/PLO/DMT/2023']
DEBUG:data_ingestion:Page 1: Detected keywords: ['062/50/PLO/DMT/2023', '063/50/PLO/DMT/2023']
[2025-11-05 11:45:57,808] DEBUG: Page 2: Detected keywords: ['062/50/PLO/DMT/2023']
DEBUG:data_ingestion:Page 2: Detected keywords: ['062/50/PLO/DMT/2023']
[2025-11-05 11:45:59,672] DEBUG: Page 3: Detected keywords: ['062/50/PLO/DMT/2023']
DEBUG:data_ingestion:Page 3: Detected keywords: ['062/50/PLO/DMT/2023']
[2025-11-05 11:46:10,375] DEBUG: Page 11: Detected keywords: ['062/50/PLO/DMT/2023']
DEBUG:data_ingestion:Page 11: Detected keywords: ['062/50/PLO/DMT/2023']
[2025-11-05 11:46:12,232] DEBUG: Page 12: Detected keywords: ['063/50/PLO/DMT/2023']
DEBUG:data_ingestion:Page 12: Detected keywords: ['063/50/PLO/DMT/2023']
[2025-11-05 11:46:13,841] DEBUG: Page 13: Detected keywords: ['063/50/PLO/DMT/2023']
DEBUG:data_ingestion:Page 13: Detected keywords: ['063/50/PLO/DMT/2023']
[2025-11-05 

In [37]:
result_indices

[0, 1, 2, 10, 11, 12, 16, 144]

In [45]:
images = convert_from_path(df['filepath'][17])
cleaned_plo_numbers = cleansing_plo_number(df['permit_number'][17])

keyword_to_pages = map_keywords_to_pages(
    images,
    anchor_keywords=cleaned_plo_numbers)    

[2025-11-05 12:23:42,145] DEBUG: Found '325/50/PLO/DMT/2024' on page 1
DEBUG:data_ingestion:Found '325/50/PLO/DMT/2024' on page 1
[2025-11-05 12:23:42,148] DEBUG: Found '326/50/PLO/DMT/2024' on page 1
DEBUG:data_ingestion:Found '326/50/PLO/DMT/2024' on page 1
[2025-11-05 12:23:42,149] DEBUG: Found '327/50/PLO/DMT/2024' on page 1
DEBUG:data_ingestion:Found '327/50/PLO/DMT/2024' on page 1
[2025-11-05 12:23:44,067] DEBUG: Found '325/50/PLO/DMT/2024' on page 2
DEBUG:data_ingestion:Found '325/50/PLO/DMT/2024' on page 2
[2025-11-05 12:23:45,593] DEBUG: Found '325/50/PLO/DMT/2024' on page 3
DEBUG:data_ingestion:Found '325/50/PLO/DMT/2024' on page 3
[2025-11-05 12:24:24,406] DEBUG: Found '325/50/PLO/DMT/2024' on page 29
DEBUG:data_ingestion:Found '325/50/PLO/DMT/2024' on page 29


In [46]:
keyword_to_pages

{'325/50/PLO/DMT/2024': [0, 1, 2, 28],
 '326/50/PLO/DMT/2024': [0],
 '327/50/PLO/DMT/2024': [0]}

In [47]:
df['filepath'][17]

'data/folder/PGN_Permits/3. PLO_PGN/PLO_SOR1-3/PLO SOR 1/PLO PGN 2023 AREA BATAM.pdf'

In [48]:
df['permit_number'][17]

array(['325/50/PLO/DMT/2024', '326/50/PLO/DMT/2024',
       '327/50/PLO/DMT/2024'], dtype=object)